# uproot 3 features

This notebook is primarily about what you can do in uproot 3 that you couldn't do in uproot 2. See the [main tutorial](https://mybinder.org/v2/gh/scikit-hep/uproot/master?filepath=binder%2Ftutorial.ipynb) for a basic introduction to uproot.

The main differences are:

   * **more modularization:** uproot is concerned purely with file I/O; functions for interacting with histograms and physics objects like TLorentzVectors have been moved to [uproot-methods](https://github.com/scikit-hep/uproot-methods). Nested data (jagged arrays) have moved to the new [awkward-array](https://github.com/scikit-hep/awkward-array) library. Also, uproot no longer has its own implementation of dict-like caches— instead, we use the third-party [cachetools](https://cachetools.readthedocs.io/en/latest/) library. As a user, this means that you can use whichever combination of versions you need: e.g. bleeding-edge object methods with a stable file I/O.
   * **jagged and object array operations:** now when you read data with structure or class definitions, you can perform Numpy-like operations on them. More on that below.
   * **writing ROOT files:** basic support for writing ROOT files has begun, with more on the way.

Let's get started!

In [1]:
import math
import numpy
import uproot
events = uproot.open("../tests/samples/HZZ-objects.root")["events"]

## Object arrays

Since uproot 2, you could read TTree branches describing class objects (defined by ROOT's streamers). This includes STL collections, TVectors, and even histograms— if you should want to store histograms in a TTree. However, it also meant leaving the high-speed Numpy world for slow pure-Python code. In uproot 3, fixed-size objects are interpreted as `awkward.ObjectArrays`, which do most operations in a vectorized way across the fields of the data.

In [2]:
MET = events.array("MET")
MET

<ArrayMethods [TVector2(5.9128, 2.5636) TVector2(24.765, -16.349) TVector2(-25.785, 16.237) ... TVector2(18.102, 50.291), TVector2(79.875, -52.351), TVector2(19.714, -3.5954)] at 7f3a300858d0>

The array above appears to contain `TVector2` objects. However, it only creates `TVector2` objects on demand (such as in the print-out). What we have actually loaded is the individual fields (`fX` and `fY`) as separate Numpy arrays.

In [3]:
MET.columns

['fX', 'fY']

In [4]:
MET["fX"]

array([  5.91277122,  24.76520348, -25.78508759, ...,  18.10164642,
        79.87519073,  19.71374893])

In [5]:
MET["fY"]

array([  2.5636332 , -16.34910965,  16.23713112, ...,  50.29071808,
       -52.35145187,  -3.59541821])

Although you can pull out any element as a `TVector2` Python object (creating it on demand),

In [6]:
MET[2], math.sqrt(MET[2].x**2 + MET[2].y**2)

(TVector2(-25.785, 16.237), 30.471546871754967)

or (gasp!) iterate over them in a Python for loop,

In [7]:
for i, met in enumerate(MET):
    print(met, math.sqrt(met.x**2 + met.y**2))
    if i > 10:
        break

TVector2(5.9128, 2.5636) 6.444616261735072
TVector2(24.765, -16.349) 29.67505163503274
TVector2(-25.785, 16.237) 30.471546871754967
TVector2(8.6199, -22.787) 24.362457116812326
TVector2(5.3931, -1.3101) 5.5499715598881885
TVector2(-3.7595, -19.417) 19.777622472715098
TVector2(23.962, -9.0492) 25.61389850143991
TVector2(-57.533, -20.488) 61.072343275467276
TVector2(42.416, -94.351) 103.44669393597131
TVector2(-1.9145, -23.963) 24.039388019581505
TVector2(19.71, 4.6455) 20.250114726294257
TVector2(-35.538, -14.754) 38.47893782676562


it's much faster to do operations across the whole array.

In [8]:
numpy.sqrt(MET.x**2 + MET.y**2)

array([ 6.44461626, 29.67505164, 30.47154687, ..., 53.4492837 ,
       95.50246389, 20.03893533])

Physics objects like `TVector2` have many methods, which are computed in Python if applied to a Python object but computed in Numpy if applied to a whole array.

In [9]:
MET[2].phi()

2.5796134389921948

In [10]:
MET.phi()

array([ 0.40911176, -0.58348763,  2.57961344, ...,  1.22529377,
       -0.58017296, -0.1803985 ])

Numpy operations apply elementwise to this array of `TVector2`, just as they would to Numpy arrays. Addition and multiplication are handled in physically meaningful ways.

In [11]:
MET + MET     # or numpy.add(MET, MET)

<ArrayMethods [TVector2(11.826, 5.1273) TVector2(49.53, -32.698) TVector2(-51.57, 32.474) ... TVector2(36.203, 100.58), TVector2(159.75, -104.7), TVector2(39.427, -7.1908)] at 7f3a30085d30>

In [12]:
MET * 2       # same output

<ArrayMethods [TVector2(11.826, 5.1273) TVector2(49.53, -32.698) TVector2(-51.57, 32.474) ... TVector2(36.203, 100.58), TVector2(159.75, -104.7), TVector2(39.427, -7.1908)] at 7f3a30085978>

In [13]:
try:
    MET + 2
except TypeError:
    print("You can't add scalars to vectors (or multiply vectors to vectors).")

You can't add scalars to vectors (or multiply vectors to vectors).


In [14]:
MET.dot(MET)

array([  41.53307876,  880.60868954,  928.51516876, ..., 2856.8259281 ,
       9120.72060822,  401.55892909])

In [15]:
MET.mag()

array([ 6.44461626, 29.67505164, 30.47154687, ..., 53.4492837 ,
       95.50246389, 20.03893533])

In [16]:
MET.delta_phi(MET)

array([0., 0., 0., ..., 0., 0., 0.])

In [17]:
MET.isparallel(MET)

array([ True,  True,  True, ...,  True,  True,  True])

## Jagged array operations

Just as in uproot 2, array-valued and `std::vector`-valued branches are presented as `JaggedArrays`. Unlike uproot 2, these are now being developed in an external library, awkward-array, with a rich set of operations that extend Numpy's built-in rules for broadcasting and indexing arrays.

In [18]:
muoniso = events.array("muoniso")
muoniso

<JaggedArray [[4.2001534 2.1510613] [2.1880474] [1.4128217 3.3835042] ... [3.7629452], [0.5508107], [0.]] at 7f3a30085ba8>

Just as with object arrays, this is not a array of thousands of one- and two-element arrays; this is efficiently stored as contiguous arrays that generate subarrays on demand.

In [19]:
muoniso[2]

array([1.4128217, 3.3835042], dtype=float32)

In [20]:
for i, event_muoniso in enumerate(muoniso):
    for particle_muoniso in event_muoniso:
        print(particle_muoniso, end="     \t")
    print()
    if i > 10:
        break

4.2001534     	2.1510613     	
2.1880474     	
1.4128217     	3.3835042     	
2.7284882     	0.5522966     	
0.0     	0.8563976     	
0.0     	1.4929442     	
0.6231756     	0.0     	
2.4025257     	
0.0     	0.0     	
0.0     	1.7698176     	
2.0015755     	0.6041591     	
0.0     	0.76338214     	


In [21]:
muoniso.content

array([4.2001534, 2.1510613, 2.1880474, ..., 3.7629452, 0.5508107,
       0.       ], dtype=float32)

In [22]:
muoniso.counts

array([2, 1, 2, ..., 1, 1, 1])

What's new is that you can manipulate jagged arrays without resorting to for loops. This makes for more succinct code (a few characters, rather than the indented body of a for loop), but it's also much faster because it is implemented in Numpy (vectorized; contiguous memory access).

In the following, we multiply each muon isolation variable by the muon charge (just an example, not physically meaningful) and maintain the structure of which muon belongs to which event.

In [23]:
muoniso * events.array("muonq")

<JaggedArray [[ 4.20015335 -2.1510613 ] [2.18804741] [ 1.41282165 -3.38350415] ... [-3.76294518], [-0.55081069], [-0.]] at 7f3a43158b38>

Naturally, the jagged structure of the two arrays must match.

In [24]:
try:
    muoniso + events.array("electroniso")
except IndexError:
    print("Not all events have the same number of electrons as muons.")

Not all events have the same number of electrons as muons.


But if you operate on a jagged array and a flat array, you can "broadcast" each event's value to all associated particles' values. Here's a simple example to illustrate that:

In [25]:
import awkward
per_particle = awkward.JaggedArray.fromiter([[1, 2, 3], [], [4, 5]])
per_event = numpy.array([100, 200, 300])

per_particle + per_event

<JaggedArray [[101 102 103] [] [304 305]] at 7f3a30050470>

100 was added to all three of the particles in the first event, 200 was added to nothing because there are no particles in the second event, and 300 was added to all particles in the third event.

As a physically meaningful example, you might want to take the phi difference between each jet in an event with the event's MET.

In [26]:
events.array("jetp4").phi() - events.array("MET").phi()

<JaggedArray [[] [3.25270257] [] ... [-2.89561452], [ 3.44895047 -1.50219424], []] at 7f3a30050630>

Or use the built-in `delta_phi` to correctly handle wrap-around.

In [27]:
events.array("jetp4").delta_phi(events.array("MET"))

<JaggedArray [[] [-3.03048274] [] ... [-2.89561452], [-2.83423484 -1.50219424], []] at 7f3a30050b00>

All but the last example involved jagged arrays of numbers. This last one combined jagged arrays with object arrays, as `delta_phi` is an object method. That's worth a new section.

## Jagged object arrays

The idea of the awkward-array library is that these special array features should be compositional: you can have jagged arrays of jagged arrays and all of the above can have object-like methods.

That is most clearly demonstrated with a jagged array of `TLorentzVectors`.

In [28]:
muonp4 = events.array("muonp4")
muonp4

<JaggedArray [[TLorentzVector(-52.899, -11.655, -8.1608, 54.779) TLorentzVector(37.738, 0.69347, -11.308, 39.402)] [TLorentzVector(-0.81646, -24.404, 20.2, 31.69)] [TLorentzVector(48.988, -21.723, 11.168, 54.74) TLorentzVector(0.82757, 29.801, 36.965, 47.489)] ... [TLorentzVector(-29.757, -15.304, -52.664, 62.395)], [TLorentzVector(1.1419, 63.61, 162.18, 174.21)], [TLorentzVector(23.913, -35.665, 54.719, 69.556)]] at 7f3a30056048>

Derived quantities like `pt` and `eta` have the same jagged structure as the `TLorentzVectors`.

In [29]:
muonp4.pt()

<JaggedArray [[54.16810703 37.74415266] [24.41791248] [53.58826697 29.81199714] ... [33.46153652], [63.61981771], [42.93994828]] at 7f3a3003c160>

In [30]:
muonp4.eta()

<JaggedArray [[-0.15009262 -0.29527553] [0.75381369] [0.20692921 1.0412953 ] ... [-1.23504687], [1.66533108], [1.06269886]] at 7f3a30050e48>

### Masking

Just as with Numpy arrays, we can apply masks to select events of interest.

In [31]:
mask = (muonp4.counts >= 2)
mask

array([ True, False,  True, ..., False, False, False])

In [32]:
muonp4[mask]

<JaggedArray [[TLorentzVector(-52.899, -11.655, -8.1608, 54.779) TLorentzVector(37.738, 0.69347, -11.308, 39.402)] [TLorentzVector(48.988, -21.723, 11.168, 54.74) TLorentzVector(0.82757, 29.801, 36.965, 47.489)] [TLorentzVector(22.088, -85.835, 403.85, 413.46) TLorentzVector(76.692, -13.956, 335.09, 344.04)] ... [TLorentzVector(53.006, -24.486, 13.952, 60.032) TLorentzVector(-30.209, 19.269, 18.661, 40.399)], [TLorentzVector(55.72, 26.37, -24.588, 66.368) TLorentzVector(-26.914, -9.8128, -0.38995, 28.65)], [TLorentzVector(34.507, 28.84, -150.66, 157.23) TLorentzVector(-31.568, -10.424, -111.26, 116.13)]] at 7f3a30050d30>

### Multidimensional indexes

Also like Numpy arrays, we can apply multidimensional indexes to pick out, say, the first and second muon in events with two muons.

In [33]:
muon0 = muonp4[mask, 0]
muon0

<ArrayMethods [TLorentzVector(-52.899, -11.655, -8.1608, 54.779) TLorentzVector(48.988, -21.723, 11.168, 54.74) TLorentzVector(22.088, -85.835, 403.85, 413.46) ... TLorentzVector(53.006, -24.486, 13.952, 60.032), TLorentzVector(55.72, 26.37, -24.588, 66.368), TLorentzVector(34.507, 28.84, -150.66, 157.23)] at 7f3a300508d0>

In [34]:
muon1 = muonp4[mask, 1]
muon1

<ArrayMethods [TLorentzVector(37.738, 0.69347, -11.308, 39.402) TLorentzVector(0.82757, 29.801, 36.965, 47.489) TLorentzVector(76.692, -13.956, 335.09, 344.04) ... TLorentzVector(-30.209, 19.269, 18.661, 40.399), TLorentzVector(-26.914, -9.8128, -0.38995, 28.65), TLorentzVector(-31.568, -10.424, -111.26, 116.13)] at 7f3a30050160>

Note that these are now non-jagged arrays. By asking for the N<sup>th</sup> in each event, we now have something that is one-per-event: non-jagged.

But they are object arrays with array methods. We can do things with them like compute masses.

In [35]:
(muon0 + muon1).mass()

array([90.22779777, 74.74654928, 89.75736376, ..., 92.06495256,
       85.44384208, 75.96066262])

### Jagged mask

An ordinary Numpy mask selects events. To select particles, make a jagged array of booleans.

In [36]:
type(muonp4.x.content)

numpy.ndarray

In [37]:
mask = (muonp4.pt() > 40)
mask

<JaggedArray [[ True False] [False] [ True False] ... [False], [ True], [ True]] at 7f3a30050be0>

In [38]:
muonp4[mask]

<JaggedArray [[TLorentzVector(-52.899, -11.655, -8.1608, 54.779)] [] [TLorentzVector(48.988, -21.723, 11.168, 54.74)] ... [], [TLorentzVector(1.1419, 63.61, 162.18, 174.21)], [TLorentzVector(23.913, -35.665, 54.719, 69.556)]] at 7f3a30050b70>

### Jagged reduction

Calling a "reducer" like `.sum()`, `.min()`, or `.max()` on a Numpy array returns a scalar. Calling the same reducer on a jagged array returns a flat array: you get the sum/min/max per event.

In [39]:
abseta = abs(muonp4.eta())
abseta

<JaggedArray [[0.15009262 0.29527553] [0.75381369] [0.20692921 1.0412953 ] ... [1.23504687], [1.66533108], [1.06269886]] at 7f3a30056240>

In [40]:
abseta.max()

array([0.29527553, 0.75381369, 1.0412953 , ..., 1.23504687, 1.66533108,
       1.06269886])

This can be useful for making event-level masks from particle data.

In [41]:
muonp4[abseta.max() < 1]    # both muons within |eta| < 1

<JaggedArray [[TLorentzVector(-52.899, -11.655, -8.1608, 54.779) TLorentzVector(37.738, 0.69347, -11.308, 39.402)] [TLorentzVector(-0.81646, -24.404, 20.2, 31.69)] [TLorentzVector(45.171, 67.249, -89.696, 120.86) TLorentzVector(39.751, 25.404, 20.115, 51.285)] ... [TLorentzVector(53.006, -24.486, 13.952, 60.032) TLorentzVector(-30.209, 19.269, 18.661, 40.399)], [TLorentzVector(55.72, 26.37, -24.588, 66.368) TLorentzVector(-26.914, -9.8128, -0.38995, 28.65)], [TLorentzVector(-24.158, -35.032, -19.194, 46.683)]] at 7f3a30056358>

### Jagged argmin/argmax

Sometimes you want to know variable `x` that is minimized/maximized over `y`. Numpy uses `argmin` and `argmax` with fancy indexing to provide that for flat arrays; we can now do the same for jagged arrays.

The `argmin` and `argmax` of a jagged array returns `[]` for empty events and `[i]` for non-empty events, where `i` is the index of the minimized/maximized item.

In [42]:
abseta.argmax()

<JaggedArray [[1] [0] [1] ... [0], [0], [0]] at 7f3a30056080>

Passing jagged indexes into the square brackets of a jagged array picks the subitems by index. Note that each event contains zero (`[]`) or one (`[TLorentzVector(...)]`) item.

In [43]:
muonp4[abseta.argmax()]

<JaggedArray [[TLorentzVector(37.738, 0.69347, -11.308, 39.402)] [TLorentzVector(-0.81646, -24.404, 20.2, 31.69)] [TLorentzVector(0.82757, 29.801, 36.965, 47.489)] ... [TLorentzVector(-29.757, -15.304, -52.664, 62.395)], [TLorentzVector(1.1419, 63.61, 162.18, 174.21)], [TLorentzVector(23.913, -35.665, 54.719, 69.556)]] at 7f3a30056588>

This would be more clear if we look at plain numbers.

In [44]:
muonp4[abseta.argmax()].pt()

<JaggedArray [[37.74415266] [24.41791248] [29.81199714] ... [33.46153652], [63.61981771], [42.93994828]] at 7f3a300567b8>

To get a min/max for only the non-empty events, you can flatten the result, which effectively concatenates all subarrays: `[]` disappear and `[x]` become `x`. (Because of our data representation, this is merely discarding the structure information, so it's instantaneous.)

In [45]:
muonp4[abseta.argmax()].flatten()

<ArrayMethods [TLorentzVector(37.738, 0.69347, -11.308, 39.402) TLorentzVector(-0.81646, -24.404, 20.2, 31.69) TLorentzVector(0.82757, 29.801, 36.965, 47.489) ... TLorentzVector(-29.757, -15.304, -52.664, 62.395), TLorentzVector(1.1419, 63.61, 162.18, 174.21), TLorentzVector(23.913, -35.665, 54.719, 69.556)] at 7f3a300562e8>

### Jagged fancy indexing

Although a major use of fancy indexing is to consume the result of an `argmin` or `argmax`, it's a general and useful technique.

In Numpy, you can pass an arbitrary sequence of indexes to an array to pick out just those elements, in the specified order. (Also useful for consuming the result of `argsort`.) If you pass such a sequence to a jagged array, you get an analogous result: selected events in the order you request them.

In [46]:
import awkward
simple = awkward.JaggedArray.fromiter([[0.0, 1.1, 2.2], [], [3.3, 4.4]])
simple[[2, 1, 2]]

<JaggedArray [[3.3 4.4] [] [3.3 4.4]] at 7f3a30056d68>

But if you pass a jagged array of indexes to a jagged array, you select particles within each event.

In [47]:
simple[awkward.JaggedArray.fromiter([[2, 1, 2], [], [1, 1, 1, 0]])]

<JaggedArray [[2.2 1.1 2.2] [] [4.4 4.4 4.4 3.3]] at 7f3a30056ba8>

This is in exact analogy with the treatment of jagged masks.

### Jagged cross-join and pairs

Numpy-like extensions let you mask particles and events, select them by index, sum/min/max over them by event, apply transformations across arrays and jagged arrays while maintaining their structure, and broadcast per-event attributes to each particle. However, none of these let you form new candidates by combining particles.

For example, say you want to loop over all jets and all muons to search for leptoquarks.

In [48]:
jetp4, muonp4 = events.arrays(["jetp4", "muonp4"], outputtype=tuple)

In [49]:
leptoquarks = []
for i in range(len(events)):
    current = []
    for jet in jetp4[i]:
        for muon in muonp4[i]:
            current.append(jet + muon)
    leptoquarks.append(current)
    if i > 10:
        break

leptoquarks

[[],
 [TLorentzVector(-39.691, -4.5408, 19.305, 75.828)],
 [],
 [TLorentzVector(-49.607, 7.7361, 600.14, 643.81),
  TLorentzVector(4.9967, 79.615, 531.39, 574.39),
  TLorentzVector(58.695, -63.997, 495.51, 514.82),
  TLorentzVector(113.3, 7.8823, 426.76, 445.4),
  TLorentzVector(-6.7781, -76.515, 455.09, 473.54),
  TLorentzVector(47.825, -4.6358, 386.34, 404.13)],
 [TLorentzVector(49.051, -7.9853, -449.3, 488.45),
  TLorentzVector(43.631, -49.83, -339.49, 418.87),
  TLorentzVector(50.151, 28.017, -21.239, 200.11),
  TLorentzVector(44.731, -13.828, 88.572, 130.53)],
 [TLorentzVector(-37.099, 67.813, 36.758, 118.54),
  TLorentzVector(-52.121, -3.0363, 94.355, 127.32),
  TLorentzVector(36.966, 26.013, -9.3573, 76.646),
  TLorentzVector(21.944, -44.837, 48.239, 85.435)],
 [TLorentzVector(-69.527, 5.1032, -386.78, 411.54),
  TLorentzVector(-52.524, 43.206, -288.69, 319.26)],
 [TLorentzVector(62.803, -48.904, 386.53, 395.3)],
 [],
 [TLorentzVector(-29.129, 44.266, -14.775, 181.23),
  TLorent

It works, but of course it's slow because of the Python for loops and the Python objects. This operation of pairing one item in set `muons` for every item in set `jets` is known in databases as a "cross-join" or a "cartesian product."

Expanding two huge sets with a cross-join would use a lot more memory than iterating over pairs, but with jagged arrays, only events need to be cross-joined. The memory use grows quadratically with the largest event, not the number of events, and subsequent operations can be vectorized (faster) on the expanded data. In many situations, it's practical to cross-join particle data.

In [52]:
pairs = jetp4.cross(muonp4)

In [51]:
pairs._0   # the first in each pair

<JaggedArray [[] [TLorentzVector(-38.875, 19.863, -0.89494, 44.137)] [] ... [TLorentzVector(-3.7148, -37.202, 41.012, 55.951)], [TLorentzVector(-36.361, 10.174, 226.43, 229.58) TLorentzVector(-15.257, -27.175, 12.12, 33.92)], []] at 7f3a30056780>

In [53]:
pairs._1   # the second in each pair

<JaggedArray [[] [TLorentzVector(-0.81646, -24.404, 20.2, 31.69)] [] ... [TLorentzVector(-29.757, -15.304, -52.664, 62.395)], [TLorentzVector(1.1419, 63.61, 162.18, 174.21) TLorentzVector(1.1419, 63.61, 162.18, 174.21)], []] at 7f3a300670f0>

In [54]:
leptoquarks = pairs._0 + pairs._1
leptoquarks

<JaggedArray [[] [TLorentzVector(-39.691, -4.5408, 19.305, 75.828)] [] ... [TLorentzVector(-33.472, -52.506, -11.652, 118.35)], [TLorentzVector(-35.219, 73.783, 388.61, 403.79) TLorentzVector(-14.115, 36.434, 174.3, 208.13)], []] at 7f3a30067d68>

### Attaching to tables

